In [2]:
import pandas as pd
import numpy as np

In [ ]:
billboard = pd.read_csv('merged_data_edited.csv', index_col=0)

In [ ]:
billboard.Song = billboard['Song'].str.lower()
billboard.Artist = billboard['Artist'].str.lower()

# Get Spotify URI

In [ ]:
from spotipy.oauth2 import SpotifyClientCredentials
from tqdm import tqdm_notebook
import spotipy
import spotipy.util as util
import re

In [ ]:
token = util.oauth2.SpotifyClientCredentials(client_id='94f6160482a043afa12209c9331b692d', client_secret='8a1f334da6324a3b836a4a0c8893ced9')
cache_token = token.get_access_token()
spotify = spotipy.Spotify(cache_token)
sp = spotipy.Spotify(auth=cache_token)

In [ ]:
# Need to get Spotify URI only for songs with missing (NaN) URI
data = billboard[billboard.URI.isna()]
data

In [ ]:
songs = list(data.Song)
artists = list(data.Artist)

In [ ]:
spotify_uri = list()
errors = list()

In [ ]:
def get_spotify_uri(title, artist):
    title_query = title
    artist_query = artist.split(' ')[0]
    query = title_query + " " + artist_query
    
    title_query_clean = re.sub(r"[,.;@#?!&$-/]+", ' ', title)
    title_query_clean = re.sub('\s+', ' ', title_query_clean).strip()
   
    artist_query_clean = re.sub(r"[,.;@#?!&$-/]+", ' ', artist)
    artist_query_clean = re.sub('\s+', ' ', artist_query_clean).strip()
    artist_query_clean = artist_query_clean.split(' ')[0]
    
#     print(title_query_clean)
#     print(artist_query_clean)
    
    search = sp.search(q=query, limit=50, offset=0, type='track', market='US')
    search_items = search['tracks']['items']
    
    for i in range(len(search_items)):
        
        spotify_title = search_items[i]['name']
        title_clean = re.sub(r"[,.;@#?!&$-/]+", ' ', spotify_title)
        title_clean = re.sub('\s+', ' ', title_clean).strip()
        title_clean = title_clean.lower()
#         print(title_clean)
        
        spotify_artist = search_items[i]['artists'][0]['name']
        artist_clean = spotify_artist.lower().split(' ')
#         print(artist_clean)
        
        if (title_query_clean in title_clean) and (artist_query_clean in artist_clean):
            uri = search_items[i]['id']
            return uri
    else:
        return 0

In [ ]:
for i in tqdm_notebook(range(len(songs))):
    uri = get_spotify_uri(songs[i], artists[i])
    
    if uri != 0:
        spotify_uri.append(uri)
    else:
        spotify_uri.append(uri)
        print("Errored on " + str(i))
        errors.append(i)    

In [ ]:
len(errors)

In [ ]:
len(spotify_uri)

In [ ]:
data['URI'] = spotify_uri

In [ ]:
data

In [ ]:
# data.to_csv('non_billboard100_uri.csv')

# Get Spotify Features

In [224]:
from spotipy.oauth2 import SpotifyClientCredentials
from tqdm import tqdm_notebook
import spotipy
import spotipy.util as util
import re

In [225]:
token = util.oauth2.SpotifyClientCredentials(client_id='94f6160482a043afa12209c9331b692d', client_secret='8a1f334da6324a3b836a4a0c8893ced9')
cache_token = token.get_access_token()
spotify = spotipy.Spotify(cache_token)
sp = spotipy.Spotify(auth=cache_token)

In [ ]:
all_data = pd.read_csv('all_data_w_uri.csv', index_col=0)

In [ ]:
all_data = all_data[all_data.URI != '0']
all_data.Song = all_data['Song'].str.lower()
all_data.Artist = all_data['Artist'].str.lower()
all_data

In [ ]:
all_uris = list(all_data.URI)

danceability_list = list()
energy_list = list()
key_list = list()
loudness_list = list()
mode_list = list()
speechiness_list = list()
acousticness_list = list()
instrumentalness_list = list()
liveness_list = list()
valence_list = list()
tempo_list = list()
duration_list = list()
time_signature_list= list()

In [ ]:
# all_uris[3481]

In [ ]:
# search = sp.audio_features('60a0Rd6pjrkxjPbaKzXjfq')
# search

In [ ]:
len(all_uris)

In [ ]:
def get_audio_features(uri):
    search = sp.audio_features(uri)
    if search[0] == None:
        danceability_list.append(np.nan)
        energy_list.append(np.nan)
        key_list.append(np.nan)
        loudness_list.append(np.nan)
        mode_list.append(np.nan)
        speechiness_list.append(np.nan)
        acousticness_list.append(np.nan)
        instrumentalness_list.append(np.nan)
        liveness_list.append(np.nan)
        valence_list.append(np.nan)
        tempo_list.append(np.nan)
        duration_list.append(np.nan)
        time_signature_list.append(np.nan) 
        return ('Error on: ' + str(uri))
    
    search_list = search[0]
    
    danceability_list.append(search_list['danceability'])
    energy_list.append(search_list['energy'])
    key_list.append(search_list['key'])
    loudness_list.append(search_list['loudness'])
    mode_list.append(search_list['mode'])
    speechiness_list.append(search_list['speechiness'])
    acousticness_list.append(search_list['acousticness'])
    instrumentalness_list.append(search_list['instrumentalness'])
    liveness_list.append(search_list['liveness'])
    valence_list.append(search_list['valence'])
    tempo_list.append(search_list['tempo'])
    duration_list.append(search_list['duration_ms'])
    time_signature_list.append(search_list['time_signature'])  

In [ ]:
for i in tqdm_notebook(range(len(all_uris))):
    get_audio_features(all_uris[i])

In [ ]:
all_data['Danceability'] = danceability_list
all_data['Energy'] = energy_list
all_data['Key'] = key_list
all_data['Loudness'] = loudness_list
all_data['Mode'] = mode_list
all_data['Speechiness'] = speechiness_list
all_data['Acousticness'] = acousticness_list
all_data['Instrumentalness'] = instrumentalness_list
all_data['Liveness'] = liveness_list
all_data['Valence'] = valence_list
all_data['Tempo'] = tempo_list
all_data['Duration'] = duration_list
all_data['Time_Signature'] = time_signature_list

In [ ]:
all_data

In [ ]:
all_data = all_data.dropna()
all_data

In [ ]:
all_data.to_csv('all_data_w_features.csv')

# Get Lyrics

In [ ]:
lyrics_data = pd.read_csv('all_data_w_features.csv', index_col=0)

In [ ]:
import requests
import json
import pandas as pd
import numpy as np
from musixmatch import Musixmatch
from tqdm import tqdm_notebook

In [ ]:
api_key = '7d8043070d8b187e54fbcb656e8537aa'
musixmatch = Musixmatch(api_key)

In [ ]:
lyrics_data['lyrics'] = np.nan
lyrics_data['explicit'] = np.nan
lyrics_data.head()

In [ ]:
lyrics_list = list()
explicit_list = list()
errors = list()

In [ ]:
def get_lyrics(track, artist):
    output = musixmatch.matcher_lyrics_get(track, artist)
    
    if (output['message']['body'] == ''):
        return None
    if (output['message']['body'] == []):
        return None
    else:
        lyrics = output['message']['body']['lyrics']['lyrics_body'].split('***')[0]
        explicit = output['message']['body']['lyrics']['explicit']
        return [lyrics, explicit]

In [ ]:
songs = list(lyrics_data.Song)
artists = list(lyrics_data.Artist)

In [ ]:
for i in tqdm_notebook(range(len(songs))):
    song = songs[i]
    artist = artists[i]
    
    output = get_lyrics(song, artist)
    
    if output == None:
        print('Errored on: ' + str(i))
        lyrics_list.append(np.nan)
        explicit_list.append(np.nan)
        errors.append(i)
    else:
        lyrics_list.append(output[0])
        explicit_list.append(output[1])   

In [ ]:
len(errors)

In [ ]:
lyrics_data.lyrics = lyrics_list
lyrics_data.explicit = explicit_list

In [ ]:
lyrics_data = lyrics_data.dropna()
lyrics_data

In [ ]:
# lyrics_data.to_csv('all_data_w_features_lyrics.csv')

# Get Genre

In [100]:
all_data_w_features = pd.read_csv('all_data_w_features_lyrics.csv', index_col=0)

In [101]:
all_data_w_features.head()

,Song,Artist,Billboard100,URI,Danceability,Energy,Key,Loudness,Mode,Speechiness,Acousticness,Instrumentalness,Liveness,Valence,Tempo,Duration,Time_Signature,lyrics,explicit
1,in the end,linkin park,0,60a0Rd6pjrkxjPbaKzXjfq,0.556,0.864,3.0,-5.870,0.0,0.0584,0.009580,0.000000,0.209,0.400,105.143,216880.0,4.0,It starts with\nOne thing I don't know why\nIt...,0.0
2,seven nation army,the white stripes,0,7i6r9KotUPQg3ozKKgEPIN,0.737,0.463,0.0,-7.828,1.0,0.0792,0.008170,0.447000,0.255,0.324,123.881,231733.0,4.0,I'm gonna fight 'em all\nA seven nation army c...,1.0
3,by the way,red hot chili peppers,0,1f2V8U1BiWaC9aJWmpOARe,0.451,0.970,0.0,-4.938,1.0,0.1070,0.026400,0.003550,0.102,0.198,122.444,216933.0,4.0,Standing in line\nTo see the show tonight\nAnd...,1.0
5,bring me to life,evanescence,0,0COqiPhxzoWICwFCS4eZcp,0.331,0.943,4.0,-3.188,0.0,0.0698,0.007210,0.000002,0.242,0.296,94.612,235893.0,4.0,"How can you see into my eyes, like open doors?...",0.0
6,last resort,papa roach,0,5W8YXBz9MTIDyrpYaCg2Ky,0.589,0.890,4.0,-3.719,0.0,0.0603,0.000481,0.000820,0.201,0.692,90.598,199907.0,4.0,Cut my life into pieces\nThis is my last resor...,1.0


In [89]:
artists = list(all_data_w_features.Artist)

In [90]:
#function to pull first artist, even if string contains &, , or 'and'
def get_first_artist(artist):
    
    #Handling , or and
    for i in range(len(artist)):
        if artist[i] == ',' or artist[i] == '&':
              return artist[0:i]
        if artist[i:i+3] == 'and':
              return artist[0:i]
    return artist

#Helper function to take subgenres of each artist and find the most frequent common genre within potentially 1000 subgenres
def get_common_genres(test_subgenres, common_genre_keywords):
    
    final_genres = []
    genre_frequency_map = {}
  
    #Checking the subgenres to see if they match common genre keywords

    for keyword in common_genre_keywords:
        for subg in test_subgenres:
            if keyword in subg:
                final_genres.append(keyword)

    #If no final genres can be identified, return None.
    if len(final_genres) == 0:
        return "None"

    #Counting the number of each genre keyword and returning the one with the highest count (as the "common genre")

    for genre in final_genres:
        if genre in genre_frequency_map.keys():
            current_value = genre_frequency_map[genre]
            genre_frequency_map[genre] = current_value + 1
        else:
            genre_frequency_map[genre] = 1

    #Getting most frequent common genre.

    max_value = max(genre_frequency_map.values())  # maximum value
    max_keys = [k for k, v in genre_frequency_map.items() if v == max_value]
    return max_keys[0]

In [91]:
def get_genres_for_each_artist(artists):
    
    final_genre_list = []
    #Most popular American Music Genres
    common_genre_keywords = ["rap", "pop", "rock", "country", "alternative" "latin", "edm", "seasonal", "jazz", "classical", "metal"]
    
    for i in tqdm_notebook(range(len(artists))):
        artist_name = artists[i]
        #Clean Artist Name 
        artist_name = get_first_artist(artist_name)
        #Include stripping code of and etc. here
 
        #stripping regex symbols and getting the prominent artist name
      
        #artist_name = re.sub(r"[,.;@#?!&$-/]+", ' ', artist_name)
        #artist_name = re.sub('\s+', ' ', artist_name).strip()
        #artist_name = artist_name(' ')[0]
 
        #Using the spotify search functionality and extracting the list of subgenres.
        try:
            search = sp.search(q=artist_name, limit=1, offset=0, type='artist', market='US')
        except Exception:
            print('Spotify error for ' + str(i))
            final_genre_list.append('None')
            continue
            

        try:
            list_of_subgenres = search['artists']['items'][0]['genres']
            common_genre = get_common_genres(list_of_subgenres, common_genre_keywords)
            final_genre_list.append(common_genre)
        except IndexError:
            print('No Genre for ' + str(i))
            final_genre_list.append('None')
            continue
    
    return final_genre_list

In [92]:
genres_list = get_genres_for_each_artist(artists)

No Genre for 8
No Genre for 213
No Genre for 288
No Genre for 301
No Genre for 437
No Genre for 609
No Genre for 711
No Genre for 984
No Genre for 1017
No Genre for 1346
No Genre for 1399
No Genre for 1518
No Genre for 1529
No Genre for 1636
No Genre for 1640
No Genre for 1651
No Genre for 1659
No Genre for 1683
No Genre for 1752
No Genre for 1792
Spotify error for 1900
No Genre for 2017
No Genre for 2034
Spotify error for 2055
Spotify error for 2231
Spotify error for 2280
No Genre for 2282
No Genre for 2348
No Genre for 2373
No Genre for 2380
No Genre for 2386
Spotify error for 2457
No Genre for 2519
No Genre for 2569
No Genre for 2593
No Genre for 2783
No Genre for 2799
No Genre for 2800
Spotify error for 2807
No Genre for 2931
No Genre for 3235
No Genre for 3492
No Genre for 3507
No Genre for 3533
No Genre for 3573
No Genre for 3612
No Genre for 3897
No Genre for 3941
No Genre for 4000
No Genre for 4001
No Genre for 4026
No Genre for 4054
No Genre for 4058
No Genre for 4084
No Genre

In [93]:
len(genres_list)

11553

In [102]:
all_data_w_features['Genre'] = genres_list

In [103]:
all_data_w_features

,Song,Artist,Billboard100,URI,Danceability,Energy,Key,Loudness,Mode,Speechiness,Acousticness,Instrumentalness,Liveness,Valence,Tempo,Duration,Time_Signature,lyrics,explicit,Genre
1,in the end,linkin park,0,60a0Rd6pjrkxjPbaKzXjfq,0.556,0.864,3.0,-5.870,0.0,0.0584,0.009580,0.000000,0.2090,0.4000,105.143,216880.0,4.0,It starts with\nOne thing I don't know why\nIt...,0.0,metal
2,seven nation army,the white stripes,0,7i6r9KotUPQg3ozKKgEPIN,0.737,0.463,0.0,-7.828,1.0,0.0792,0.008170,0.447000,0.2550,0.3240,123.881,231733.0,4.0,I'm gonna fight 'em all\nA seven nation army c...,1.0,rock
3,by the way,red hot chili peppers,0,1f2V8U1BiWaC9aJWmpOARe,0.451,0.970,0.0,-4.938,1.0,0.1070,0.026400,0.003550,0.1020,0.1980,122.444,216933.0,4.0,Standing in line\nTo see the show tonight\nAnd...,1.0,rock
5,bring me to life,evanescence,0,0COqiPhxzoWICwFCS4eZcp,0.331,0.943,4.0,-3.188,0.0,0.0698,0.007210,0.000002,0.2420,0.2960,94.612,235893.0,4.0,"How can you see into my eyes, like open doors?...",0.0,metal
6,last resort,papa roach,0,5W8YXBz9MTIDyrpYaCg2Ky,0.589,0.890,4.0,-3.719,0.0,0.0603,0.000481,0.000820,0.2010,0.6920,90.598,199907.0,4.0,Cut my life into pieces\nThis is my last resor...,1.0,metal
9,sex on fire,kings of leon,0,0ntQJM78wzOLVeCUAW7Y45,0.542,0.905,9.0,-5.653,1.0,0.0540,0.001720,0.010400,0.1360,0.3740,153.398,203347.0,4.0,Lay where you're laying\nDon't make a sound\nI...,1.0,rock
12,smooth criminal,alien ant farm,0,5z6xHjCZr7a7AIcy8sPBKy,0.653,0.964,9.0,-4.261,0.0,0.0582,0.003160,0.005120,0.1440,0.8700,126.928,209267.0,4.0,As he came into the window\nWas a sound of a c...,0.0,metal
14,chop suey!,system of a down,0,2DlHlPMa4M17kufBvI2lEN,0.419,0.934,7.0,-3.908,0.0,0.1200,0.000278,0.001500,0.1320,0.2860,127.288,210240.0,4.0,Wake up\n(wake up)\nGrab a brush and put a lit...,0.0,metal
15,take me out,franz ferdinand,0,6ooluO7DiEhI1zmK94nRCM,0.278,0.676,9.0,-8.821,1.0,0.0371,0.000423,0.000899,0.1360,0.4940,104.545,237027.0,4.0,"So if you're lonely, you know I'm here waiting...",0.0,None
20,the kill (bury me),thirty seconds to mars,0,232FvPwx9QqPtvob5aSDKv,0.303,0.926,4.0,-3.945,0.0,0.0695,0.001150,0.002350,0.2750,0.2720,182.915,233427.0,3.0,What if I wanted to break?\nLaugh it all off i...,0.0,rock


In [104]:
# Dropping 'None' from Genre

all_data_w_features = all_data_w_features[all_data_w_features['Genre'] != 'None']

In [105]:
all_data_w_features

,Song,Artist,Billboard100,URI,Danceability,Energy,Key,Loudness,Mode,Speechiness,Acousticness,Instrumentalness,Liveness,Valence,Tempo,Duration,Time_Signature,lyrics,explicit,Genre
1,in the end,linkin park,0,60a0Rd6pjrkxjPbaKzXjfq,0.556,0.864,3.0,-5.870,0.0,0.0584,0.009580,0.000000,0.2090,0.4000,105.143,216880.0,4.0,It starts with\nOne thing I don't know why\nIt...,0.0,metal
2,seven nation army,the white stripes,0,7i6r9KotUPQg3ozKKgEPIN,0.737,0.463,0.0,-7.828,1.0,0.0792,0.008170,0.447000,0.2550,0.3240,123.881,231733.0,4.0,I'm gonna fight 'em all\nA seven nation army c...,1.0,rock
3,by the way,red hot chili peppers,0,1f2V8U1BiWaC9aJWmpOARe,0.451,0.970,0.0,-4.938,1.0,0.1070,0.026400,0.003550,0.1020,0.1980,122.444,216933.0,4.0,Standing in line\nTo see the show tonight\nAnd...,1.0,rock
5,bring me to life,evanescence,0,0COqiPhxzoWICwFCS4eZcp,0.331,0.943,4.0,-3.188,0.0,0.0698,0.007210,0.000002,0.2420,0.2960,94.612,235893.0,4.0,"How can you see into my eyes, like open doors?...",0.0,metal
6,last resort,papa roach,0,5W8YXBz9MTIDyrpYaCg2Ky,0.589,0.890,4.0,-3.719,0.0,0.0603,0.000481,0.000820,0.2010,0.6920,90.598,199907.0,4.0,Cut my life into pieces\nThis is my last resor...,1.0,metal
9,sex on fire,kings of leon,0,0ntQJM78wzOLVeCUAW7Y45,0.542,0.905,9.0,-5.653,1.0,0.0540,0.001720,0.010400,0.1360,0.3740,153.398,203347.0,4.0,Lay where you're laying\nDon't make a sound\nI...,1.0,rock
12,smooth criminal,alien ant farm,0,5z6xHjCZr7a7AIcy8sPBKy,0.653,0.964,9.0,-4.261,0.0,0.0582,0.003160,0.005120,0.1440,0.8700,126.928,209267.0,4.0,As he came into the window\nWas a sound of a c...,0.0,metal
14,chop suey!,system of a down,0,2DlHlPMa4M17kufBvI2lEN,0.419,0.934,7.0,-3.908,0.0,0.1200,0.000278,0.001500,0.1320,0.2860,127.288,210240.0,4.0,Wake up\n(wake up)\nGrab a brush and put a lit...,0.0,metal
20,the kill (bury me),thirty seconds to mars,0,232FvPwx9QqPtvob5aSDKv,0.303,0.926,4.0,-3.945,0.0,0.0695,0.001150,0.002350,0.2750,0.2720,182.915,233427.0,3.0,What if I wanted to break?\nLaugh it all off i...,0.0,rock
22,no one knows,queens of the stone age,0,6NvRxjfYkkT2SpirAlmsjH,0.511,0.581,0.0,-4.930,1.0,0.0581,0.031000,0.000171,0.3670,0.6680,170.852,278707.0,4.0,"We get some rules to follow\nThat and this, th...",0.0,rock


In [108]:
# all_data_w_features.to_csv('all_data_w_new_features.csv')

# Get Year

In [226]:
data = pd.read_csv('all_data_w_new_features.csv', index_col=0)
data.head()

,Song,Artist,Billboard100,URI,Danceability,Energy,Key,Loudness,Mode,Speechiness,Acousticness,Instrumentalness,Liveness,Valence,Tempo,Duration,Time_Signature,lyrics,explicit,Genre
1,in the end,linkin park,0,60a0Rd6pjrkxjPbaKzXjfq,0.556,0.864,3.0,-5.870,0.0,0.0584,0.009580,0.000000,0.209,0.400,105.143,216880.0,4.0,It starts with\nOne thing I don't know why\nIt...,0.0,metal
2,seven nation army,the white stripes,0,7i6r9KotUPQg3ozKKgEPIN,0.737,0.463,0.0,-7.828,1.0,0.0792,0.008170,0.447000,0.255,0.324,123.881,231733.0,4.0,I'm gonna fight 'em all\nA seven nation army c...,1.0,rock
3,by the way,red hot chili peppers,0,1f2V8U1BiWaC9aJWmpOARe,0.451,0.970,0.0,-4.938,1.0,0.1070,0.026400,0.003550,0.102,0.198,122.444,216933.0,4.0,Standing in line\nTo see the show tonight\nAnd...,1.0,rock
5,bring me to life,evanescence,0,0COqiPhxzoWICwFCS4eZcp,0.331,0.943,4.0,-3.188,0.0,0.0698,0.007210,0.000002,0.242,0.296,94.612,235893.0,4.0,"How can you see into my eyes, like open doors?...",0.0,metal
6,last resort,papa roach,0,5W8YXBz9MTIDyrpYaCg2Ky,0.589,0.890,4.0,-3.719,0.0,0.0603,0.000481,0.000820,0.201,0.692,90.598,199907.0,4.0,Cut my life into pieces\nThis is my last resor...,1.0,metal


In [227]:
songs = list(data.Song)
artists = list(data.Artist)
years = list()
errors = list()

In [228]:
def get_song_year(title, artist):
    title_query = title
    artist_query = artist
    query = title_query + " " + artist_query
    
    title_query_clean = re.sub(r"[,.;@#?!&$-/]+", ' ', title)
    title_query_clean = re.sub('\s+', ' ', title_query_clean).strip()
   
    artist_query_clean = re.sub(r"[,.;@#?!&$-/]+", ' ', artist)
    artist_query_clean = re.sub('\s+', ' ', artist_query_clean).strip()

    try:
        search = sp.search(q=query, limit=50, offset=0, type='track')
        search_items = search['tracks']['items']
        year = search_items[0]['album']['release_date']
        return year
    except Exception:
        year = 0
        return year

In [229]:
for i in tqdm_notebook(range(len(songs))):
    year = get_song_year(songs[i], artists[i])
    
    if year != 0 :
        years.append(year)
    else:
        years.append(year)
        print("Errored on " + str(i))
        errors.append(i)    

Errored on 172
retrying ...1secs
retrying ...1secs
Errored on 1606
retrying ...1secs
Errored on 2471
Errored on 3570
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
Errored on 7069
Errored on 7118
Errored on 7552
Errored on 7557
Errored on 7567
Errored on 7580
Errored on 7657
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs


In [252]:
new_years = list()

def clean_year(date):
    y = date.split('-')[0]
    return int(y)

In [253]:
for i in years:
    if i == 0:
        new_years.append(i)
    else:
        x = clean_year(i)
        new_years.append(x)

In [255]:
data['Year'] = new_years

In [258]:
data = data[data.Year != 0]

In [263]:
data = data[data.Year < 2019]

In [264]:
data = data[data.Year >= 1990]

In [265]:
data

,Song,Artist,Billboard100,URI,Danceability,Energy,Key,Loudness,Mode,Speechiness,...,Instrumentalness,Liveness,Valence,Tempo,Duration,Time_Signature,lyrics,explicit,Genre,Year
1,in the end,linkin park,0,60a0Rd6pjrkxjPbaKzXjfq,0.556,0.864,3.0,-5.870,0.0,0.0584,...,0.000000,0.2090,0.4000,105.143,216880.0,4.0,It starts with\nOne thing I don't know why\nIt...,0.0,metal,2000
2,seven nation army,the white stripes,0,7i6r9KotUPQg3ozKKgEPIN,0.737,0.463,0.0,-7.828,1.0,0.0792,...,0.447000,0.2550,0.3240,123.881,231733.0,4.0,I'm gonna fight 'em all\nA seven nation army c...,1.0,rock,2003
3,by the way,red hot chili peppers,0,1f2V8U1BiWaC9aJWmpOARe,0.451,0.970,0.0,-4.938,1.0,0.1070,...,0.003550,0.1020,0.1980,122.444,216933.0,4.0,Standing in line\nTo see the show tonight\nAnd...,1.0,rock,2002
5,bring me to life,evanescence,0,0COqiPhxzoWICwFCS4eZcp,0.331,0.943,4.0,-3.188,0.0,0.0698,...,0.000002,0.2420,0.2960,94.612,235893.0,4.0,"How can you see into my eyes, like open doors?...",0.0,metal,2003
6,last resort,papa roach,0,5W8YXBz9MTIDyrpYaCg2Ky,0.589,0.890,4.0,-3.719,0.0,0.0603,...,0.000820,0.2010,0.6920,90.598,199907.0,4.0,Cut my life into pieces\nThis is my last resor...,1.0,metal,2001
9,sex on fire,kings of leon,0,0ntQJM78wzOLVeCUAW7Y45,0.542,0.905,9.0,-5.653,1.0,0.0540,...,0.010400,0.1360,0.3740,153.398,203347.0,4.0,Lay where you're laying\nDon't make a sound\nI...,1.0,rock,2008
12,smooth criminal,alien ant farm,0,5z6xHjCZr7a7AIcy8sPBKy,0.653,0.964,9.0,-4.261,0.0,0.0582,...,0.005120,0.1440,0.8700,126.928,209267.0,4.0,As he came into the window\nWas a sound of a c...,0.0,metal,2001
14,chop suey!,system of a down,0,2DlHlPMa4M17kufBvI2lEN,0.419,0.934,7.0,-3.908,0.0,0.1200,...,0.001500,0.1320,0.2860,127.288,210240.0,4.0,Wake up\n(wake up)\nGrab a brush and put a lit...,0.0,metal,2001
20,the kill (bury me),thirty seconds to mars,0,232FvPwx9QqPtvob5aSDKv,0.303,0.926,4.0,-3.945,0.0,0.0695,...,0.002350,0.2750,0.2720,182.915,233427.0,3.0,What if I wanted to break?\nLaugh it all off i...,0.0,rock,2007
22,no one knows,queens of the stone age,0,6NvRxjfYkkT2SpirAlmsjH,0.511,0.581,0.0,-4.930,1.0,0.0581,...,0.000171,0.3670,0.6680,170.852,278707.0,4.0,"We get some rules to follow\nThat and this, th...",0.0,rock,2002


In [266]:
# data.to_csv('FINAL_DATA_2.csv')